# Digital EMG Signal Feature Extraction

Import packages.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from filter import BandpassFilter1D, NotchFilter1D
from processing import MeanShift1D, Detrend1D, Resample1D, Normalize1D
from segment import Segment1D, Segment2D
from visualize import plot_signal1d, plot_signalnd
%matplotlib inline

EMG signal processing functions

In [ ]:
# process signal of each channel
def process_signal1d(x, raw_fs=1000, low_fs=1, high_fs=120, notch_fs=60, Q=20, window_size=250, step_size=50, target_fs=512):
    """
    @param x: signal of a single channel
    @param raw_fs: original sampling rate
    @param low_fs: low cutoff frequency
    @param high_fs: high cutoff frequency
    @param notch_fs: notch cutoff frequency
    @param Q: Q factor
    @param window_size: windows size for detrending
    @param step_size: step size for detrending
    @param target_fs: target sampling rate for resampling step
    """
    # mean-correct signal
    x_processed = MeanShift1D.apply(x)
    # filtering noise
    x_processed = BandpassFilter1D.apply(x_processed, low_fs, high_fs, order=4, fs=raw_fs)
    x_processed = NotchFilter1D.apply(x_processed, notch_fs, Q=Q, fs=raw_fs)
    # detrend
    x_processed = Detrend1D.apply(x_processed, detrend_type='locreg', window_size=window_size, step_size=step_size)
    # resample
    x_processed = Resample1D.apply(x_processed, raw_fs, target_fs)
    # rectify
    x_processed = abs(x_processed)
    # normalize
    x_processed = Normalize1D.apply(x_processed, norm_type='min_max')
    return x_processed

In [ ]:
# process multi-channel signal
def process_signalnd(x, num_channels, raw_fs=1000, low_fs=1, high_fs=120, notch_fs=60, Q=20, window_size=250, step_size=50, target_fs=512):
    """
    @param x: signal of a single channel
    @param raw_fs: original sampling rate
    @param low_fs: low cutoff frequency
    @param high_fs: high cutoff frequency
    @param notch_fs: notch cutoff frequency
    @param Q: Q factor
    @param window_size: windows size for detrending
    @param step_size: step size for detrending
    @param target_fs: target sampling rate for resampling step
    """
    x_processed = []
    for i in range(num_channels):
        channel_processed = process_signal1d(x[:, i], raw_fs, low_fs, high_fs, notch_fs, Q, window_size, step_size, target_fs)
        x_processed.append(channel_processed)
    # convert emg signal to numpy matrix
    x_processed = np.array(x_processed)
    return x_processed

## EMG Feature Extraction

With the aim to facilitate the classification process, the raw signal should be transformed into relevant data structure or feature vector to highlight the important data for pattern recognition. There are three types of features in EMG control system: 1) time domain, 2) frequency domain, 3) time-frequency domain.

### 1. Time Domain Features

The time domain features are the most popular in EMG pattern recognition, because they are easy and quick to calculate as well as they do not require transformation. The time domain features usually are computed based on signal amplitude, and resultant values.

##### Signal Power

This feature is computed by calculating the summation of absolute values of signal.

$$\text{PD}_{k}=\sum_{i=1}^{N}{|x_{i}|}$$

where $x_{i}$ is value of signal segment $k$, $N$ is the length of the signal segment.

##### Mean Absolute (MA)

This feature is computed by adding the absolute value of all values in a signal segment and dividing it by the length segment.

$$\text{MA}_{k}=\frac{1}{N}\sum_{i=1}^{N}{|x_{i}|}$$

where $x_{i}$ is value of signal segment $k$, $N$ is the length of the signal segment.

##### Root Mean Square (RMS)

It is modeled as amplitude modulated Gaussian random process whose RMS is related to the constant force and non-fatiguing contraction.

$$\text{RMS}_{k}=\sqrt{\frac{1}{N}\sum_{i=1}^{N}{x_{i}^{2}}}$$

where $x_{i}$ is value of signal segment $k$, $N$ is the length of the signal segment.

##### Variance (VAR)

$$\text{VAR}_{k}=\sqrt{\frac{1}{N}\sum_{i=1}^{N}{(x_{i}-\bar{x})^{2}}}$$

where $x_{i}$ is value of signal segment $k$, \bar{x} is mean value of signal segment, $N$ is the length of the signal segment.

##### Waveform Length (WL)

It is the cumulative length of the waveform over the segment. The resultant values indicate a measure of waveform amplitude, frequency and duration all within a single parameter.

$$\text{WL}_{k}=\sum_{i=1}^{N-1}{|x_{i+1}-x_{i}|}$$

where $x_{i}$ and $x_{i+1}$ is consecutive values of signal segment $k$, $N$ is the length of the signal segment.

##### William Amplitude (WAMP)

It calculates the number of times that the absolute value of the difference between EMG signal amplitude of two consecutive samples ($x_{i}$ and $x_{i+1}$) exceeds a predetermined threshold $\epsilon$.

$$\text{WAMP}_{k}=\sum_{i=1}^{N-1}{f(x_{i}-x_{i+1})}$$

$$f(x)=\begin{cases}{1}&&{x>\epsilon}\\{0}&&{\text{otherwise}\end{cases}$$

where $x_{i}$ and $x_{i+1}$ is consecutive values of signal segment $k$, $N$ is the length of the signal segment, \epsilon is threshold value.

##### Simple Square Integral (SSI).

This feature uses the energy of signal.

$$\text{SSI}_{k}=\sum_{i=1}^{N}{|x_{i}^{2}|}$$

where $x_{i}$ is value of signal segment $k$, $N$ is the length of the signal segment.

### 2. Frequency Domain Features

The frequency domain features are based on estimated Power Spectrum Density (PSD) of signal and are computed by periodogram or parametric methods. In comparison with time domain features, frequency domain features requires more computation.

##### Fequency Median (FMD)

The frequency median splits the power specturm density into two equal parts.

$$\text{FMD}=\frac{1}{2}\sum_{i=1}^{N}{\text{PSD}_{i}}$$

where $N$ is the length of the power spectrum density, and $\text{PSD}_{i}$ is the $i^{th}$ line of power spectrum density.

##### Frequency Mean (FMN)

$$\text{FMN}=\frac{\sum_{i=1}^{M}{f_{i}\text{PSD}_{i}}}{\sum_{i=1}^{M}{\text{PSD}_{i}}}$$

where $N$ is the length of the power spectrum density, and $\text{PSD}_{i}$ is the $i^{th}$ line of power spectrum density.